# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

The analytics team at Sparkify wants to analyze the data they've been collecting on songs and user activity on their new music streaming app. Currently, the data resided in a directory of csv files partitioned by date. The following steps in Part I will create a single denormalized dataset to prepare it for data modeling and analysis with Apache Cassandra database.

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Processing dataset into Apache Cassandra Database

### From Part I, a CSV file titled <font color=red>event_datafile_new.csv</font> should be located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Connecting to Apache Cassandra

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create tables to run the following queries on Apache Cassandra
(_with Apache Cassandra we will model the database tables based on the queries we want to run_)

#### The analytics team wants to answer these three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

We are interested in searching the dataset by both sessionId and itemInSession. The sessionId would make a good choice for the partition key becase of its low cardinality, as this will determine the data distribution across the system. The itemInSession, because of it's high cardinality within a sessionId, can be included as a clustering column to sort the data within the partition. Combined, these 2 keys should make a unique primary key. 

In [8]:
query = "CREATE TABLE IF NOT EXISTS songplay_sessions "
query = query + "(sessionid int, iteminsession int, artist_name text, song_title text, length float, PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songplay_sessions (sessionid, iteminsession, artist_name, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verify that the data have been inserted into the table and is modeled to retrieve data for the stated query

In [10]:
query = "SELECT artist_name, song_title, length FROM songplay_sessions WHERE sessionId=338 AND iteminsession=4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

We are interested in searching the dataset by both userid and sessionId, and then retrieving results sorted by itemInSession. The userid and sessionId would make a good choice for a composite partition key, as this will determine the data distribution. The itemInSession can be included as a clustering column, which would be used to sort the data. Combined, these 3 keys should make a unique primary key. 

In [11]:
query = "CREATE TABLE IF NOT EXISTS user_songplay_sessions "
query = query + "(userid int, sessionId int, itemInSession int, first_name text, last_name text, artist_name text, song_title text, PRIMARY KEY ((userid, sessionId), iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)             

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_songplay_sessions (userid, sessionId, iteminsession, first_name, last_name, artist_name, song_title)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

#### Verify that the data have been inserted into the table and is modeled to retrieve data for the stated query

In [13]:
query = "SELECT artist_name, song_title, first_name, last_name, iteminsession FROM user_songplay_sessions WHERE userid=10 AND sessionId=182 ORDER BY iteminsession;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name, row.iteminsession)

Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'    
We are interested in searching the dataset by song title. The song title would make a good choice for a partition key, as this will determine the data distribution. However, it is not enough to be unique. Since we are interested in users, the user's name and userid can be included as a composite clustering column, which would be used to sort the data. Combined, these 4 keys should make a unique primary key for the intention of the query (there may be duplicates of the same userid listening to the same song removed). 

In [14]:
query = "CREATE TABLE IF NOT EXISTS user_songplay "
query = query + "(song_title text, artist_name text, first_name text, last_name text, userid int, PRIMARY KEY (song_title, last_name, first_name, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)         

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songplay (song_title, first_name, last_name, userid)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

#### Verify that the data have been inserted into the table and is modeled to retrieve data for the stated query

In [16]:
query = "SELECT first_name, last_name FROM user_songplay WHERE song_title = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Sara Johnson
Tegan Levine
Jacqueline Lynch


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions

query = "DROP TABLE IF EXISTS songplay_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_songplay_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_songplay"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()